In [1]:
import json
from pathlib import Path
from operator import itemgetter
import pandas as pd

In [2]:
# List all model directories
ray_results = Path("../ray_results/")
model_dirs = [d for d in ray_results.iterdir() if d.is_dir()]

In [3]:
# Function to find best checkpoints for a model directory
def find_best_checkpoints(model_dir, num_best=10):
    checkpoints = []
    
    # Iterate over all training runs in the model directory
    for run_dir in model_dir.iterdir():
        if run_dir.is_dir():
            # Read the progress.csv file to get the validation losses
            progress_file = run_dir / "progress.csv"
            if progress_file.exists():
                with open(run_dir / "params.json", "r") as f:
                    params = json.load(f)

                progress_data = pd.read_csv(progress_file)
                best_val_idx = progress_data["val_loss"].idxmin()
                best_val_loss = progress_data.loc[best_val_idx, "val_loss"]
                
                # Save the checkpoint path and validation loss
                checkpoint_path = run_dir / "my_model" / "checkpoint.pt"
                checkpoints.append((checkpoint_path, best_val_loss, params))
    
    # Sort the checkpoints based on validation loss
    checkpoints.sort(key=itemgetter(1))
    
    return checkpoints[:num_best]

# Find the best 10 checkpoints for each model and store the results in DataFrames
model_dfs = {}
# Find the best 10 checkpoints for each model
for model_dir in model_dirs:
    rows = []
    best_checkpoints = find_best_checkpoints(model_dir)
    for i, (checkpoint, val_loss, params) in enumerate(best_checkpoints):
        #print(f"{i+1}. Model {params['model']} (val_loss: {val_loss}, params: {params})")
        # Save the model, validation loss, and variables as a row
        rows.append({
            "model": params["model"],
            "val_loss": val_loss,
            "variables": params['data']["variables"],
            })
    print()
    # Create a DataFrame from the rows and sort it based on validation loss
    df = pd.DataFrame(rows).sort_values("val_loss")
    model_dfs[model_dir.name] = df

In [4]:
model_dfs.keys()

dict_keys(['data_1-inflow_forecast', 'data_2-inflow_forecast', 'data_3-inflow_forecast', 'data_4-inflow_forecast'])

In [5]:
# set the display option to show full content of variables column
pd.set_option('max_colwidth', 5000)

In [6]:
model_dfs.get("data_1-inflow_forecast")

,model,val_loss,variables
0,LSTM,0.070786,"[Wind_Speed_Nilsebu, Water_Level_Kalltveit, Temperature_HBV, Relative_Humidity_Nilsebu, Flow_HBV, Flow_Tapping, Precipitation_Fister, Air_Temperature_Nilsebu, Water_Temperature_Lyngsaana, Water_Temperature_Kalltveit_Kum]"
1,LSTM,0.072178,"[Wind_Speed_Nilsebu, Water_Level_Kalltveit, Temperature_HBV, Relative_Humidity_Nilsebu, Flow_HBV, Flow_Tapping, Precipitation_Fister, Air_Temperature_Nilsebu, Water_Temperature_Lyngsaana, Water_Temperature_Kalltveit_Kum]"
2,LSTM,0.072250,"[Wind_Speed_Nilsebu, Water_Level_Kalltveit, Temperature_HBV, Relative_Humidity_Nilsebu, Flow_HBV, Flow_Tapping, Precipitation_Fister, Air_Temperature_Nilsebu, Water_Temperature_Lyngsaana, Water_Temperature_Kalltveit_Kum]"
3,LSTM,0.073341,"[Wind_Speed_Nilsebu, Water_Level_Kalltveit, Temperature_HBV, Relative_Humidity_Nilsebu, Flow_HBV, Flow_Tapping, Precipitation_Fister, Air_Temperature_Nilsebu, Water_Temperature_Lyngsaana, Water_Temperature_Kalltveit_Kum]"
4,LSTMTemporalAttention,0.073871,"[Precipitation_Fister, Flow_Lyngsvatn_Overflow, Water_Temperature_Lyngsaana]"
5,LSTM,0.075178,"[Wind_Speed_Nilsebu, Water_Level_Kalltveit, Temperature_HBV, Relative_Humidity_Nilsebu, Flow_HBV, Flow_Tapping, Precipitation_Fister, Air_Temperature_Nilsebu, Water_Temperature_Lyngsaana, Water_Temperature_Kalltveit_Kum]"
6,LSTM,0.075827,"[Wind_Speed_Nilsebu, Water_Level_Kalltveit, Temperature_HBV, Relative_Humidity_Nilsebu, Flow_HBV, Flow_Tapping, Precipitation_Fister, Air_Temperature_Nilsebu, Water_Temperature_Lyngsaana, Water_Temperature_Kalltveit_Kum]"
7,LSTMTemporalAttention,0.076513,"[Flow_Lyngsvatn_Overflow, Air_Temperature_Nilsebu, Water_Temperature_Lyngsaana, Wind_Direction_Nilsebu, Temperature_HBV, Relative_Humidity_Nilsebu, Water_Temperature_Kalltveit_Kum, Flow_Without_Tapping_Kalltveit, Precipitation_Fister, Wind_Speed_Nilsebu, Precipitation_Nilsebu, Flow_Tapping, Flow_HBV, Water_Level_Kalltveit, Precipitation_HBV]"
8,LSTM,0.081690,"[Wind_Speed_Nilsebu, Water_Level_Kalltveit, Temperature_HBV, Relative_Humidity_Nilsebu, Flow_HBV, Flow_Tapping, Precipitation_Fister, Air_Temperature_Nilsebu, Water_Temperature_Lyngsaana, Water_Temperature_Kalltveit_Kum]"
9,LSTM,0.081730,"[Wind_Speed_Nilsebu, Water_Level_Kalltveit, Temperature_HBV, Relative_Humidity_Nilsebu, Flow_HBV, Flow_Tapping, Precipitation_Fister, Air_Temperature_Nilsebu, Water_Temperature_Lyngsaana, Water_Temperature_Kalltveit_Kum]"


In [7]:
model_dfs.get("data_2-inflow_forecast")

,model,val_loss,variables
0,LSTMTemporalAttention,0.171103,"[Water_Level_Hiafossen, Wind_Direction_Nilsebu, Precipitation_Fister, Air_Temperature_Fister, Flow_Without_Tapping_Kalltveit]"
1,LSTMTemporalAttention,0.174324,"[Water_Level_Hiafossen, Wind_Direction_Nilsebu, Precipitation_Fister, Air_Temperature_Fister, Flow_Without_Tapping_Kalltveit]"
2,LSTMTemporalAttention,0.176062,"[Relative_Humidity_Nilsebu, Wind_Direction_Nilsebu, Flow_Lyngsaana, Flow_Tapping, Precipitation_HBV]"
3,LSTMTemporalAttention,0.186678,"[Flow_HBV, Water_Level_Lyngsaana, Precipitation_Fister, Water_Level_Kalltveit, Relative_Humidity_Nilsebu, Water_Temperature_Hiafossen, Water_Level_Hiafossen, Precipitation_Nilsebu, Air_Temperature_Nilsebu, Water_Temperature_Lyngsaana]"
4,LSTM,0.187695,"[Water_Level_Hiafossen, Wind_Direction_Nilsebu, Precipitation_Fister, Air_Temperature_Fister, Flow_Without_Tapping_Kalltveit]"
5,LSTMSpatialTemporalAttention,0.190262,"[Flow_HBV, Water_Level_Lyngsaana, Precipitation_Fister, Water_Level_Kalltveit, Relative_Humidity_Nilsebu, Water_Temperature_Hiafossen, Water_Level_Hiafossen, Precipitation_Nilsebu, Air_Temperature_Nilsebu, Water_Temperature_Lyngsaana]"
6,LSTMTemporalAttention,0.190598,"[Flow_Tapping, Water_Temperature_Hiafossen, Flow_Lyngsvatn_Overflow, Water_Level_Lyngsaana, Wind_Direction_Nilsebu, Flow_HBV, Water_Temperature_Kalltveit_Kum, Relative_Humidity_Nilsebu, Precipitation_HBV, Air_Temperature_Nilsebu, Air_Temperature_Fister, Water_Temperature_Lyngsaana, Wind_Speed_Nilsebu, Water_Level_Kalltveit, Temperature_HBV, Flow_Lyngsaana]"
7,LSTMTemporalAttention,0.192002,"[Wind_Speed_Nilsebu, Flow_Lyngsaana, Flow_Tapping, Temperature_HBV, Water_Level_Hiafossen, Water_Level_Lyngsaana, Flow_Lyngsvatn_Overflow, Precipitation_HBV, Air_Temperature_Nilsebu, Air_Temperature_Fister, Precipitation_Nilsebu, Wind_Direction_Nilsebu, Water_Temperature_Hiafossen, Precipitation_Fister, Flow_Without_Tapping_Kalltveit, Relative_Humidity_Nilsebu]"
8,LSTMTemporalAttention,0.192116,"[Flow_HBV, Water_Level_Lyngsaana, Precipitation_Fister, Water_Level_Kalltveit, Relative_Humidity_Nilsebu, Water_Temperature_Hiafossen, Water_Level_Hiafossen, Precipitation_Nilsebu, Air_Temperature_Nilsebu, Water_Temperature_Lyngsaana]"
9,LSTMTemporalAttention,0.192138,"[Wind_Speed_Nilsebu, Flow_Lyngsaana, Flow_Tapping, Temperature_HBV, Water_Level_Hiafossen, Water_Level_Lyngsaana, Flow_Lyngsvatn_Overflow, Precipitation_HBV, Air_Temperature_Nilsebu, Air_Temperature_Fister, Precipitation_Nilsebu, Wind_Direction_Nilsebu, Water_Temperature_Hiafossen, Precipitation_Fister, Flow_Without_Tapping_Kalltveit, Relative_Humidity_Nilsebu]"


In [8]:
model_dfs.get("data_3-inflow_forecast")

,model,val_loss,variables
0,LSTMTemporalAttention,0.185830,"[Water_Temperature_Lyngsaana, Precipitation_HBV, Flow_HBV, Water_Temperature_Musdalsvatn_Downstream, Water_Level_Musdalsvatn, Flow_Tapping, Precipitation_Nilsebu, Water_Level_Hiavatn, Water_Level_Viglesdalsvatn, Flow_Lyngsvatn_Overflow, Water_Temperature_Kalltveit_Kum, Relative_Humidity_Nilsebu, Water_Temperature_Viglesdalsvatn, Air_Temperature_Fister, Water_Level_Kalltveit, Water_Temperature_Hiavatn, Precipitation_Fister, Temperature_HBV, Water_Level_Musdalsvatn_Downstream, Flow_Lyngsaana, Wind_Direction_Nilsebu, Water_Level_Hiafossen, Water_Temperature_Hiafossen, Air_Temperature_Nilsebu, Water_Level_Lyngsaana, Flow_Without_Tapping_Kalltveit, Wind_Speed_Nilsebu, Water_Temperature_Musdalsvatn]"
1,LSTMTemporalAttention,0.191661,"[Flow_Lyngsaana, Flow_Lyngsvatn_Overflow, Water_Level_Hiafossen, Precipitation_Nilsebu, Flow_Tapping, Water_Level_Musdalsvatn, Relative_Humidity_Nilsebu, Wind_Direction_Nilsebu]"
2,LSTMTemporalAttention,0.193436,"[Flow_Lyngsaana, Flow_Lyngsvatn_Overflow, Water_Level_Hiafossen, Precipitation_Nilsebu, Flow_Tapping, Water_Level_Musdalsvatn, Relative_Humidity_Nilsebu, Wind_Direction_Nilsebu]"
3,LSTMTemporalAttention,0.193441,"[Flow_Lyngsaana, Flow_Lyngsvatn_Overflow, Water_Level_Hiafossen, Precipitation_Nilsebu, Flow_Tapping, Water_Level_Musdalsvatn, Relative_Humidity_Nilsebu, Wind_Direction_Nilsebu]"
4,LSTMSpatialTemporalAttention,0.193640,"[Water_Level_Kalltveit, Flow_HBV, Wind_Speed_Nilsebu, Water_Level_Hiavatn, Water_Temperature_Kalltveit_Kum, Water_Temperature_Viglesdalsvatn, Wind_Direction_Nilsebu]"
5,LSTMTemporalAttention,0.196645,"[Water_Level_Hiafossen, Flow_Lyngsaana, Water_Level_Kalltveit, Water_Level_Lyngsaana, Precipitation_HBV, Water_Level_Hiavatn, Temperature_HBV, Flow_Lyngsvatn_Overflow, Air_Temperature_Nilsebu, Flow_Without_Tapping_Kalltveit, Wind_Speed_Nilsebu, Relative_Humidity_Nilsebu, Precipitation_Fister, Water_Temperature_Kalltveit_Kum, Water_Temperature_Hiavatn, Water_Temperature_Hiafossen, Water_Level_Viglesdalsvatn, Water_Temperature_Lyngsaana, Flow_HBV, Water_Level_Musdalsvatn, Air_Temperature_Fister, Water_Temperature_Viglesdalsvatn]"
6,LSTMTemporalAttention,0.199363,[Water_Temperature_Musdalsvatn]
7,LSTMTemporalAttention,0.200171,"[Flow_Lyngsaana, Flow_Lyngsvatn_Overflow, Water_Level_Hiafossen, Precipitation_Nilsebu, Flow_Tapping, Water_Level_Musdalsvatn, Relative_Humidity_Nilsebu, Wind_Direction_Nilsebu]"
8,LSTMTemporalAttention,0.200544,"[Water_Temperature_Musdalsvatn, Water_Level_Hiafossen, Water_Temperature_Hiavatn, Flow_HBV, Water_Temperature_Musdalsvatn_Downstream, Water_Temperature_Hiafossen, Flow_Tapping, Water_Temperature_Kalltveit_Kum, Water_Level_Viglesdalsvatn, Wind_Direction_Nilsebu, Precipitation_Nilsebu, Temperature_HBV, Water_Temperature_Lyngsaana, Flow_Without_Tapping_Kalltveit, Precipitation_Fister, Water_Level_Hiavatn, Relative_Humidity_Nilsebu, Flow_Lyngsaana, Water_Level_Musdalsvatn, Flow_Lyngsvatn_Overflow, Air_Temperature_Nilsebu, Precipitation_HBV, Water_Level_Lyngsaana, Water_Temperature_Viglesdalsvatn, Water_Level_Kalltveit]"
9,LSTMTemporalAttention,0.200773,"[Water_Temperature_Musdalsvatn, Water_Level_Hiafossen, Water_Temperature_Hiavatn, Flow_HBV, Water_Temperature_Musdalsvatn_Downstream, Water_Temperature_Hiafossen, Flow_Tapping, Water_Temperature_Kalltveit_Kum, Water_Level_Viglesdalsvatn, Wind_Direction_Nilsebu, Precipitation_Nilsebu, Temperature_HBV, Water_Temperature_Lyngsaana, Flow_Without_Tapping_Kalltveit, Precipitation_Fister, Water_Level_Hiavatn, Relative_Humidity_Nilsebu, Flow_Lyngsaana, Water_Level_Musdalsvatn, Flow_Lyngsvatn_Overflow, Air_Temperature_Nilsebu, Precipitation_HBV, Water_Level_Lyngsaana, Water_Temperature_Viglesdalsvatn, Water_Level_Kalltveit]"


In [9]:
model_dfs.get("data_4-inflow_forecast")

,model,val_loss,variables
0,LSTMSpatialTemporalAttention,0.172537,"[Precipitation_Fister, Water_Level_Musdalsvatn, Flow_Without_Tapping_Kalltveit, Flow_Tapping, Precipitation_Nilsebu, Water_Temperature_Musdalsvatn_Downstream, Water_Temperature_Musdalsvatn, Water_Temperature_Lyngsaana, Water_Temperature_Hiavatn, Water_Level_Lyngsaana, Water_Level_Hiavatn, Water_Level_Musdalsvatn_Downstream, Wind_Direction_Nilsebu, Relative_Humidity_Nilsebu, Flow_Lyngsaana, Air_Temperature_Fister, Water_Level_Kalltveit, Water_Level_Viglesdalsvatn, Flow_Lyngsvatn_Overflow, Flow_HBV, Water_Temperature_Kalltveit_Kum, Precipitation_HBV]"
1,LSTM,0.175960,"[Water_Temperature_Musdalsvatn_Downstream, Water_Temperature_Lyngsaana, Temperature_HBV, Air_Temperature_Fister, Water_Level_Hiafossen, Water_Level_Kalltveit, Water_Temperature_Viglesdalsvatn, Air_Temperature_Nilsebu, Relative_Humidity_Nilsebu, Water_Temperature_Kalltveit_Kum, Water_Level_Musdalsvatn_Downstream, Precipitation_Fister, Water_Temperature_Musdalsvatn, Water_Level_Hiavatn, Water_Level_Musdalsvatn, Precipitation_HBV, Flow_Tapping, Water_Temperature_Hiavatn, Water_Level_Viglesdalsvatn, Flow_Lyngsaana, Wind_Speed_Nilsebu, Precipitation_Nilsebu]"
2,LSTMSpatialTemporalAttention,0.177271,"[Precipitation_Fister, Water_Level_Musdalsvatn, Flow_Without_Tapping_Kalltveit, Flow_Tapping, Precipitation_Nilsebu, Water_Temperature_Musdalsvatn_Downstream, Water_Temperature_Musdalsvatn, Water_Temperature_Lyngsaana, Water_Temperature_Hiavatn, Water_Level_Lyngsaana, Water_Level_Hiavatn, Water_Level_Musdalsvatn_Downstream, Wind_Direction_Nilsebu, Relative_Humidity_Nilsebu, Flow_Lyngsaana, Air_Temperature_Fister, Water_Level_Kalltveit, Water_Level_Viglesdalsvatn, Flow_Lyngsvatn_Overflow, Flow_HBV, Water_Temperature_Kalltveit_Kum, Precipitation_HBV]"
3,LSTMTemporalAttention,0.184858,"[Water_Temperature_Hiavatn, Water_Level_Lyngsaana, Precipitation_Fister, Wind_Speed_Nilsebu, Water_Level_Viglesdalsvatn, Flow_Tapping, Water_Temperature_Musdalsvatn, Water_Level_Hiafossen, Water_Level_Hiavatn, Flow_Without_Tapping_Kalltveit, Air_Temperature_Nilsebu, Precipitation_HBV, Relative_Humidity_Nilsebu, Temperature_HBV, Flow_Lyngsvatn_Overflow, Water_Temperature_Hiafossen, Water_Temperature_Musdalsvatn_Downstream, Water_Temperature_Kalltveit_Kum, Wind_Direction_Nilsebu, Water_Level_Musdalsvatn_Downstream, Flow_Lyngsaana, Water_Temperature_Viglesdalsvatn, Precipitation_Nilsebu, Water_Temperature_Lyngsaana, Flow_HBV, Water_Level_Kalltveit, Air_Temperature_Fister, Water_Level_Musdalsvatn]"
4,LSTM,0.185906,"[Water_Temperature_Musdalsvatn_Downstream, Water_Temperature_Lyngsaana, Temperature_HBV, Air_Temperature_Fister, Water_Level_Hiafossen, Water_Level_Kalltveit, Water_Temperature_Viglesdalsvatn, Air_Temperature_Nilsebu, Relative_Humidity_Nilsebu, Water_Temperature_Kalltveit_Kum, Water_Level_Musdalsvatn_Downstream, Precipitation_Fister, Water_Temperature_Musdalsvatn, Water_Level_Hiavatn, Water_Level_Musdalsvatn, Precipitation_HBV, Flow_Tapping, Water_Temperature_Hiavatn, Water_Level_Viglesdalsvatn, Flow_Lyngsaana, Wind_Speed_Nilsebu, Precipitation_Nilsebu]"
5,LSTMSpatialTemporalAttention,0.187835,"[Precipitation_Fister, Water_Level_Musdalsvatn, Flow_Without_Tapping_Kalltveit, Flow_Tapping, Precipitation_Nilsebu, Water_Temperature_Musdalsvatn_Downstream, Water_Temperature_Musdalsvatn, Water_Temperature_Lyngsaana, Water_Temperature_Hiavatn, Water_Level_Lyngsaana, Water_Level_Hiavatn, Water_Level_Musdalsvatn_Downstream, Wind_Direction_Nilsebu, Relative_Humidity_Nilsebu, Flow_Lyngsaana, Air_Temperature_Fister, Water_Level_Kalltveit, Water_Level_Viglesdalsvatn, Flow_Lyngsvatn_Overflow, Flow_HBV, Water_Temperature_Kalltveit_Kum, Precipitation_HBV]"
6,LSTMSpatialTemporalAttention,0.189386,"[Precipitation_Fister, Water_Level_Musdalsvatn, Flow_Without_Tapping_Kalltveit, Flow_Tapping, Precipitation_Nilsebu, Water_Temperature_Musdalsvatn_Downstream, Water_Temperature_Musdalsvatn, Water_Temperature_Lyngsaana, Water_Temperature_Hiavatn, Wa

In [10]:
# set the display option to default
pd.set_option('max_colwidth', 50)